In [ ]:
%pip install sdv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Run this code if not mounted

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# importing necessary libraries
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
import pandas as pd
import numpy as np

RuntimeError: ignored

ImportError: ignored

## Helper functions

In [ ]:

def create_nir_with_response(df):
    """
    Function that selects columns that have complete spectra, and adds Dry Matter 
    as target. It then returns the dataframe with rows permuted.
    """
    nir = df.loc[:, '699':'1032'].copy()
    nir["DM"] = df["DM"].copy()
    return nir.reindex(np.random.RandomState(seed=1).permutation(nir.index))
    

### Autoencoder

### Loading data

In [ ]:
mango_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Master/Mango_nir_vis.csv")
mango_data

In [ ]:
nir = create_nir_with_response(mango_data)
X = nir.copy()
y = X.pop("DM")

We see that the spectras are quite smooth in the beginning, but get more tacky towards the end

### Creating metadata

In [ ]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=nir)

RuntimeError: ignored

ImportError: ignored

### Setting up CTGAN

In [ ]:

ctgan_synthesizer = CTGANSynthesizer(
    metadata, # required
    epochs=50,
    batch_size=50,
    verbose=True,
    discriminator_lr=2e-5,
    generator_lr=2e-4
)

### Training CTGAN

In [ ]:
ctgan_synthesizer.fit(X)

Epoch 1, Loss G:  2.5106,Loss D: -2.4491
Epoch 2, Loss G:  0.2869,Loss D: -2.5451
Epoch 3, Loss G: -0.3004,Loss D: -4.1487
Epoch 4, Loss G: -0.4357,Loss D: -7.3829
Epoch 5, Loss G: -0.5497,Loss D: -1.5362
Epoch 6, Loss G: -0.8313,Loss D: -4.4750
Epoch 7, Loss G: -2.6245,Loss D: -1.0207
Epoch 8, Loss G: -3.0518,Loss D: -2.5710
Epoch 9, Loss G: -2.5245,Loss D: -4.4988
Epoch 10, Loss G: -0.2570,Loss D:  2.0560
Epoch 11, Loss G: -2.0141,Loss D: -2.2042
Epoch 12, Loss G: -3.3758,Loss D: -1.1963
Epoch 13, Loss G: -2.1499,Loss D:  0.1430
Epoch 14, Loss G: -0.8306,Loss D:  0.3369
Epoch 15, Loss G: -2.1305,Loss D:  0.3484
Epoch 16, Loss G: -0.7039,Loss D: -3.1940
Epoch 17, Loss G: -1.6685,Loss D:  1.4887
Epoch 18, Loss G: -2.8381,Loss D:  1.4565
Epoch 19, Loss G: -3.2403,Loss D: -1.1546
Epoch 20, Loss G:  1.1485,Loss D: -3.5314
Epoch 21, Loss G: -0.9418,Loss D:  2.7236
Epoch 22, Loss G:  0.2435,Loss D:  0.0313
Epoch 23, Loss G:  0.7629,Loss D: -0.6390
Epoch 24, Loss G: -1.8338,Loss D: -3.1104
E

### Creating and saving dataset

In [ ]:
# Creating dataframe with synthetic samples
ctgan_nir = ctgan_synthesizer.sample(len(nir))

In [ ]:
# saving dataframe as csv-file
ctgan_nir.to_csv("/content/drive/MyDrive/Colab Notebooks/Master/ctgan_nir.csv", index=False)